<a href="https://colab.research.google.com/github/bhargavgadhiyaNarayana/MonitorManager/blob/main/BS4_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [89]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [90]:
url = 'https://www.imdb.com/chart/top/'

In [91]:
HEADERS = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }
res = requests.get(url,headers=HEADERS)

In [96]:
html_content = res.text
soup = BeautifulSoup(html_content,"html.parser")
list_of_movie_list =  soup.find_all('li',{'class':'ipc-metadata-list-summary-item sc-1364e729-0 caNpAE cli-parent'})

In [ ]:
base_url = "https://www.imdb.com"
# mov_list = list()
for movie_li in list_of_movie_list[:251]:
  mov_detail_link = movie_li.find('a',{'class':'ipc-title-link-wrapper'}).get('href')
  durl = base_url + mov_detail_link
  res_detail = requests.get(durl,headers=HEADERS)
  detail_html_content = res_detail.text
  soup_detail = BeautifulSoup(detail_html_content,"html.parser")
  # mov_name = soup_detail.find('span',{'data-testid':'hero__primary-text'}).text
  # mov_meta_data = list()
  # for meta_data in soup_detail.find('ul',{'class':'ipc-inline-list ipc-inline-list--show-dividers sc-d8941411-2 cdJsTz baseAlt'}).children:
  #   mov_meta_data.append(meta_data.text)
  # mov_genre = list()
  # for genre_list in soup_detail.find('div',{'class':'ipc-chip-list__scroller'}).children:
  #   mov_genre.append(genre_list.text)
  # mov_genre = "|".join(mov_genre)
  # mov_rating = soup_detail.find("span",{'class':'sc-bde20123-1 cMEQkK'}).text
  # mov_description = soup_detail.find("span",{'data-testid':'plot-xl'}).text
  for mov_director in soup_detail.find("li",{'data-testid':'title-pc-principal-credit'}).children:
    print(mov_director)

  # print(mov_name, mov_meta_data, mov_genre, mov_rating, mov_description)
  break
  # mov_list.append(mov_name)
# print(mov_list[0])

In [ ]:
with open(f"Imdb_top_250_movies_meta_data_{datetime.now()}.csv","a+") as f:
  f.write("sep=;\n")
  f.write("Movie Name;Movie_Year;Movie_Duration;Movie_Rating;Movie_Public_Rating;Movie_Rating_Count\n")
  for movie_li in list_of_movie_list[:251]:
    mov_name = ((movie_li.find_all('h3',{'class':'ipc-title__text'})[0].text).replace('.','|',1)).split('|')[1]
    mov_rating, mov_rating_count = movie_li.find('span',{'data-testid':'ratingGroup--imdb-rating'}).text.split("\xa0")
    mov_meta_data=list()
    for meta_data in movie_li.find('div',{'class','sc-be6f1408-7 iUtHEN cli-title-metadata'}).children:
      mov_meta_data.append(meta_data.text)
    if(len(mov_meta_data) == 2): mov_meta_data.append("Not Rated")
    f.write(f"{mov_name.strip()};{mov_meta_data[0]};{mov_meta_data[1]};{mov_meta_data[2]};{mov_rating};{mov_rating_count}\n")
  f.close()